# ノートブック18: House Prices回帰 - GBDT回帰の完全マスター 🏠

**学習目標**: Kaggle House PricesコンペティションでTop 20%（RMSLE < 0.13）を達成

---

## 📋 このノートブックで学ぶこと

### 1. 回帰特有の課題 ⭐⭐⭐
- Target transformation（対数変換、Box-Cox変換）
- Outlier detection（IsolationForest、Z-score）
- Skewness correction（歪度の補正）
- Heavy-tailed distributions（裾の重い分布）

### 2. 大規模特徴量エンジニアリング ⭐⭐⭐
- 200+特徴量の作成
- Polynomial features（多項式特徴量）
- Target encoding（ターゲットエンコーディング）
- Feature selection（特徴量選択）

### 3. GBDT回帰モデリング ⭐⭐⭐
- Objective functions: RMSE, MAE, Huber
- Quantile regression（分位点回帰）
- Feature interactions in trees

### 4. モデル評価と診断 ⭐⭐
- RMSLE（Root Mean Squared Log Error）
- Residual analysis（残差分析）
- Prediction interval estimation

---

## 🎯 目標スコア

- **Local CV RMSLE**: 0.12-0.13
- **Public LB RMSLE**: < 0.13 (Top 20%)

---

In [ ]:
# 基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from scipy import stats
from scipy.special import boxcox1p
from scipy.stats import skew, kurtosis

# 機械学習
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import mutual_info_regression

# GBDT
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

# 設定
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.max_columns', None)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ セットアップ完了")

## 📊 1. データ読み込みと基本的なEDA

In [ ]:
# データディレクトリ
data_dir = Path('../data/house_prices')
data_dir.mkdir(parents=True, exist_ok=True)

try:
    train = pd.read_csv(data_dir / 'train.csv')
    test = pd.read_csv(data_dir / 'test.csv')
    print("✅ データ読み込み成功")
except FileNotFoundError:
    print("⚠️ デモ用データを生成（実際にはKaggleからダウンロード必要）")
    # デモ用のダミーデータ
    from sklearn.datasets import make_regression
    X, y = make_regression(n_samples=1460, n_features=80, n_informative=60, 
                           noise=10, random_state=42)
    train = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(80)])
    train['SalePrice'] = np.exp(y / 100) * 100000  # 価格をリアルに
    train['Id'] = range(1, 1461)
    test = pd.DataFrame(X[:1459], columns=[f'feature_{i}' for i in range(80)])
    test['Id'] = range(1461, 2920)

print(f"\nTrain shape: {train.shape}")
print(f"Test shape: {test.shape}")

if 'SalePrice' in train.columns:
    print(f"\n🏠 価格統計:")
    print(train['SalePrice'].describe())
    
    # 価格の分布
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    sns.histplot(train['SalePrice'], kde=True, ax=axes[0])
    axes[0].set_title('Sale Price Distribution')
    axes[0].set_xlabel('Price')
    
    stats.probplot(train['SalePrice'], plot=axes[1])
    axes[1].set_title('Q-Q Plot')
    
    plt.tight_layout()
    plt.show()
    
    # 歪度と尖度
    print(f"\nSkewness: {skew(train['SalePrice']):.4f}")
    print(f"Kurtosis: {kurtosis(train['SalePrice']):.4f}")

## 🔨 2. Target Transformation（対数変換） ⭐⭐⭐

価格は対数正規分布に従うことが多いため、対数変換を適用します。

In [ ]:
if 'SalePrice' in train.columns:
    # 元の価格を保存
    train['SalePrice_original'] = train['SalePrice']
    
    # 対数変換
    train['SalePrice'] = np.log1p(train['SalePrice'])
    
    print("✅ Target transformation完了")
    print(f"変換前 - Skewness: {skew(train['SalePrice_original']):.4f}")
    print(f"変換後 - Skewness: {skew(train['SalePrice']):.4f}")
    
    # 変換後の分布
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.histplot(train['SalePrice'], kde=True, ax=ax)
    ax.set_title('Log-Transformed Sale Price')
    ax.set_xlabel('log(Price)')
    plt.show()

## 🔍 3. Outlier Detection（外れ値検出） ⭐⭐⭐

In [ ]:
def detect_outliers_isolation_forest(df, contamination=0.05):
    """
    Isolation Forestで外れ値を検出
    """
    # 数値カラムのみ
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # 欠損値を埋める
    df_numeric = df[numeric_cols].fillna(df[numeric_cols].median())
    
    # Isolation Forest
    iso_forest = IsolationForest(contamination=contamination, random_state=RANDOM_STATE)
    outliers = iso_forest.fit_predict(df_numeric)
    
    return outliers == -1

# 外れ値検出
outliers = detect_outliers_isolation_forest(train, contamination=0.05)
print(f"検出された外れ値: {outliers.sum()} 件 ({outliers.sum()/len(train)*100:.2f}%)")

# 外れ値を除去（オプション）
train_clean = train[~outliers].copy()
print(f"\nクリーニング後のサイズ: {train_clean.shape[0]}")

## 🎨 4. 高度な特徴量エンジニアリング ⭐⭐⭐

In [ ]:
def advanced_feature_engineering_house(df, is_train=True):
    """
    House Prices用の高度な特徴量エンジニアリング
    """
    df = df.copy()
    
    # 1. 総面積の計算（存在する場合）
    if all(col in df.columns for col in ['GrLivArea', '1stFlrSF', '2ndFlrSF']):
        df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df.get('TotalBsmtSF', 0)
        df['TotalBathrooms'] = (df.get('FullBath', 0) + 
                                0.5 * df.get('HalfBath', 0) + 
                                df.get('BsmtFullBath', 0) + 
                                0.5 * df.get('BsmtHalfBath', 0))
        df['TotalPorchSF'] = (df.get('OpenPorchSF', 0) + 
                              df.get('EnclosedPorch', 0) + 
                              df.get('3SsnPorch', 0) + 
                              df.get('ScreenPorch', 0))
    
    # 2. 築年数と改築年
    if 'YearBuilt' in df.columns:
        df['Age'] = 2024 - df['YearBuilt']
        if 'YearRemodAdd' in df.columns:
            df['AgeRemod'] = 2024 - df['YearRemodAdd']
            df['IsRemodeled'] = (df['YearBuilt'] != df['YearRemodAdd']).astype(int)
    
    # 3. 品質スコア（存在する場合）
    if 'OverallQual' in df.columns and 'OverallCond' in df.columns:
        df['QualityScore'] = df['OverallQual'] * df['OverallCond']
    
    # 4. 地下室の有無
    if 'TotalBsmtSF' in df.columns:
        df['HasBasement'] = (df['TotalBsmtSF'] > 0).astype(int)
    
    # 5. ガレージの有無と面積
    if 'GarageArea' in df.columns:
        df['HasGarage'] = (df['GarageArea'] > 0).astype(int)
    
    # 6. 交互作用特徴量
    if 'GrLivArea' in df.columns and 'OverallQual' in df.columns:
        df['QualArea'] = df['GrLivArea'] * df['OverallQual']
    
    # 7. 比率特徴量
    if 'GrLivArea' in df.columns and 'LotArea' in df.columns:
        df['LivingAreaRatio'] = df['GrLivArea'] / (df['LotArea'] + 1)  # 0除算回避
    
    print(f"✅ 特徴量エンジニアリング完了: {df.shape[1]} 特徴量")
    return df

# 特徴量エンジニアリングの適用
train_fe = advanced_feature_engineering_house(train_clean)
test_fe = advanced_feature_engineering_house(test)

print(f"\nTrain shape: {train_fe.shape}")
print(f"Test shape: {test_fe.shape}")

### 4.1 歪度の補正（Skewness Correction） ⭐⭐⭐

In [ ]:
def fix_skewness(train_df, test_df, threshold=0.75):
    """
    歪度が大きい特徴量にBox-Cox変換を適用
    """
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns
    
    # 歪度を計算
    skewed_features = train_df[numeric_cols].apply(lambda x: skew(x.dropna()))
    skewed_features = skewed_features[abs(skewed_features) > threshold]
    
    print(f"✅ {len(skewed_features)} 個の歪んだ特徴量を検出")
    
    # Box-Cox変換
    for feat in skewed_features.index:
        if feat in train_df.columns and feat in test_df.columns:
            # 負の値がある場合はスキップ
            if train_df[feat].min() < 0 or test_df[feat].min() < 0:
                continue
            
            # Box-Cox変換（lambda=0.15）
            train_df[feat] = boxcox1p(train_df[feat], 0.15)
            test_df[feat] = boxcox1p(test_df[feat], 0.15)
    
    return train_df, test_df

# 歪度補正
train_fe, test_fe = fix_skewness(train_fe, test_fe, threshold=0.75)
print("✅ 歪度補正完了")

## 🎯 5. データ準備

In [ ]:
def prepare_regression_data(train_df, test_df):
    """
    回帰モデル用のデータ準備
    """
    # 使用しない列
    drop_cols = ['Id', 'SalePrice', 'SalePrice_original']
    
    # 特徴量とターゲット
    if 'SalePrice' in train_df.columns:
        y = train_df['SalePrice']
    else:
        y = None
    
    X = train_df.drop(columns=[col for col in drop_cols if col in train_df.columns])
    X_test = test_df.drop(columns=[col for col in drop_cols if col in test_df.columns])
    
    # カテゴリカル変数のエンコーディング
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    
    for col in categorical_cols:
        le = LabelEncoder()
        combined = pd.concat([X[col], X_test[col]], ignore_index=True)
        le.fit(combined.astype(str))
        X[col] = le.transform(X[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))
    
    # 欠損値補完
    X = X.fillna(X.median())
    X_test = X_test.fillna(X_test.median())
    
    print(f"✅ データ準備完了")
    print(f"   特徴量数: {X.shape[1]}")
    print(f"   訓練データ: {X.shape[0]}")
    print(f"   テストデータ: {X_test.shape[0]}")
    
    return X, y, X_test

X, y, X_test = prepare_regression_data(train_fe, test_fe)

## 🚀 6. GBDT回帰モデリング

### 6.1 RMSLE評価関数の定義

In [ ]:
def rmsle(y_true, y_pred):
    """
    Root Mean Squared Log Error
    """
    return np.sqrt(mean_squared_error(y_true, y_pred))

def rmsle_cv(model, X, y, n_folds=5):
    """
    K-Fold CVでRMSLEを計算
    """
    kf = KFold(n_folds, shuffle=True, random_state=RANDOM_STATE)
    rmse_scores = -cross_val_score(model, X, y, 
                                    scoring='neg_root_mean_squared_error', 
                                    cv=kf)
    return rmse_scores

print("✅ 評価関数定義完了")

### 6.2 LightGBM回帰

In [ ]:
# LightGBMパラメータ（回帰用）
params_lgb = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 1000,
    'max_depth': 6,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': RANDOM_STATE,
    'verbose': -1
}

model_lgb = lgb.LGBMRegressor(**params_lgb)

# Cross-validation
scores_lgb = rmsle_cv(model_lgb, X, y, n_folds=5)
print(f"\n📊 LightGBM CV RMSLE: {scores_lgb.mean():.4f} (+/- {scores_lgb.std():.4f})")

# 全データで学習
model_lgb.fit(X, y)
preds_lgb = model_lgb.predict(X_test)

### 6.3 XGBoost回帰

In [ ]:
# XGBoostパラメータ
params_xgb = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 1000,
    'min_child_weight': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'reg_alpha': 0.1,
    'reg_lambda': 1,
    'random_state': RANDOM_STATE
}

model_xgb = xgb.XGBRegressor(**params_xgb)

scores_xgb = rmsle_cv(model_xgb, X, y, n_folds=5)
print(f"\n📊 XGBoost CV RMSLE: {scores_xgb.mean():.4f} (+/- {scores_xgb.std():.4f})")

model_xgb.fit(X, y)
preds_xgb = model_xgb.predict(X_test)

### 6.4 CatBoost回帰

In [ ]:
# CatBoostパラメータ
params_cat = {
    'loss_function': 'RMSE',
    'depth': 6,
    'learning_rate': 0.05,
    'iterations': 1000,
    'l2_leaf_reg': 3,
    'random_state': RANDOM_STATE,
    'verbose': False
}

model_cat = CatBoostRegressor(**params_cat)

scores_cat = rmsle_cv(model_cat, X, y, n_folds=5)
print(f"\n📊 CatBoost CV RMSLE: {scores_cat.mean():.4f} (+/- {scores_cat.std():.4f})")

model_cat.fit(X, y)
preds_cat = model_cat.predict(X_test)

## 🎭 7. アンサンブル

In [ ]:
# 重み付きアンサンブル
weights = [0.4, 0.3, 0.3]  # LightGBM, XGBoost, CatBoost

ensemble_preds = (weights[0] * preds_lgb + 
                  weights[1] * preds_xgb + 
                  weights[2] * preds_cat)

print(f"\n🎯 アンサンブル重み: LGB={weights[0]}, XGB={weights[1]}, CAT={weights[2]}")

# 対数変換を元に戻す
final_predictions = np.expm1(ensemble_preds)

print(f"\n📊 予測価格統計:")
print(f"  Mean: ${final_predictions.mean():,.0f}")
print(f"  Median: ${np.median(final_predictions):,.0f}")
print(f"  Min: ${final_predictions.min():,.0f}")
print(f"  Max: ${final_predictions.max():,.0f}")

## 📊 8. 残差分析

In [ ]:
# 訓練データでの予測
train_preds = (weights[0] * model_lgb.predict(X) + 
               weights[1] * model_xgb.predict(X) + 
               weights[2] * model_cat.predict(X))

residuals = y - train_preds

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 残差のヒストグラム
axes[0, 0].hist(residuals, bins=50, edgecolor='black')
axes[0, 0].set_title('Residuals Distribution')
axes[0, 0].set_xlabel('Residual')
axes[0, 0].set_ylabel('Frequency')

# 2. 残差 vs 予測値
axes[0, 1].scatter(train_preds, residuals, alpha=0.5)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_title('Residuals vs Predicted')
axes[0, 1].set_xlabel('Predicted')
axes[0, 1].set_ylabel('Residual')

# 3. Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot')

# 4. 実測値 vs 予測値
axes[1, 1].scatter(y, train_preds, alpha=0.5)
axes[1, 1].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
axes[1, 1].set_title('Actual vs Predicted')
axes[1, 1].set_xlabel('Actual')
axes[1, 1].set_ylabel('Predicted')

plt.tight_layout()
plt.show()

# R²スコア
r2 = r2_score(y, train_preds)
print(f"\n📊 R² Score: {r2:.4f}")

## 📤 9. Kaggle提出ファイル作成

In [ ]:
# 提出ファイル作成
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': final_predictions
})

submission_path = submission_dir / 'house_prices_submission.csv'
submission.to_csv(submission_path, index=False)

print(f"✅ 提出ファイル作成: {submission_path}")
print(f"\n最初の10行:")
display(submission.head(10))

## 🎓 10. まとめ

### 学んだこと

1. ✅ Target transformation（対数変換）の重要性
2. ✅ Outlier detection（Isolation Forest）
3. ✅ Skewness correction（Box-Cox変換）
4. ✅ GBDT回帰モデル（LightGBM、XGBoost、CatBoost）
5. ✅ RMSLE評価指標
6. ✅ 残差分析による診断

### 次のステップ

- **ノートブック19**: 時系列予測×GBDT
- **ノートブック20**: Optunaで最適化をさらに深く学ぶ

---

**🏠 Happy House Hunting! 🎉**